In [1]:
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
import pickle
from utils.dataset_func import encode
import cv2


Using TensorFlow backend.


In [2]:
FILE_PATH = 'Dataset/dataset_test.pkl'
with open (FILE_PATH, 'rb') as f:
    dataset_test = pickle.load(f)
X_test = dataset_test.texts.values
y_test = dataset_test.likes.values
id_test = dataset_test.id.values


X_test = encode(X_test)
X_img = np.empty((len(id_test), 224, 224, 3))
for i, ID in enumerate(id_test):
    img = cv2.imread('Dataset/images/'+ID)
    img = cv2.resize(img,(224,224))
    img = img/255
    #print(img)
    X_img[i, :, :, :] = img

100%|██████████| 1533/1533 [00:00<00:00, 3745.17it/s]


In [3]:
DIR = 'saved_models/'
MODEL_IMAGE_NAME = 'image_model_v1.h5' 
MODEL_TEXT_NAME = 'text_model_v1.h5'
model_text = keras.models.load_model(DIR+MODEL_TEXT_NAME)
model_image = keras.models.load_model(DIR+MODEL_IMAGE_NAME)

/home/ludapak/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:1252: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [4]:
np.array([X_test[0]]).shape

(1, 300)

In [10]:
y_pred_text = model_text.predict(X_test[:10])  
y_pred_image = model_image.predict(X_img[:10])

y_pred = (y_pred_text+y_pred_image)/2
y_pred

array([[ 0.09992398],
       [ 0.08718137],
       [ 0.09791075],
       [ 0.10081579],
       [ 0.0912454 ],
       [ 0.08601106],
       [ 0.09758849],
       [ 0.09816303],
       [ 0.10465217],
       [ 0.09997354]], dtype=float32)

In [13]:
y_test[:10]

array([ 0.04797168,  0.05608903,  0.0845506 ,  0.10660374,  0.04410628,
        0.1926598 ,  0.10345038,  0.20120438,  0.11954266,  0.05435977])

In [11]:
print(y_test)
mse_t = mean_squared_error(y_test, y_pred_text)
mse_i = mean_squared_error(y_test, y_pred_image)
mse = mean_squared_error(y_test, y_pred)

mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)
print(mse_t, mse_i, mse)
print(mae)
print(medae)

[ 0.04797168  0.05608903  0.0845506  ...,  0.03371038  0.10426415
  0.09661472]


ValueError: Found input variables with inconsistent numbers of samples: [1533, 10]